In [1]:
import time
import requests
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [2]:
# Configuration du WebDriver
driver = webdriver.Chrome()
url = "https://www.trustpilot.com/categories/atm"
driver.get(url)
wait = WebDriverWait(driver, 5)
time.sleep(2)

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [3]:
try:
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'onetrust-close-btn-handler'))
    )
    
    button.click()
    print("Bouton cliqué avec succès.")
    
except Exception as e:
    print("Impossible de cliquer sur le bouton :", e)

Bouton cliqué avec succès.


In [4]:
div_elements = driver.find_elements(By.XPATH, '//div[@class="paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2"]')
result_list = []
for div in div_elements:
    elements = div.find_element(By.CLASS_NAME, 'styles_footer__DoJVj')
    elements_2 = elements.find_element(By.CLASS_NAME, 'styles_wrapper___E6__')
    span_elements = elements_2.find_elements(By.TAG_NAME, 'span')
    concatenated_text = '' 
    for span in span_elements:
        concatenated_text += span.text 

    result_list.append(concatenated_text)
df_new_column = pd.DataFrame(result_list, columns=['infos'])

In [5]:
informations_entreprises = []

# Récupérer tous les éléments div avec la classe styles_businessUnitMain__PuwB7
entreprise_elements = driver.find_elements(By.CLASS_NAME, 'styles_businessUnitMain__PuwB7')

next_page_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
next_page_link = next_page_button.get_attribute('href')

driver.execute_script("window.open('{}', '_blank');".format(next_page_link))
driver.switch_to.window(driver.window_handles[-1])
entreprise_elements_2 = driver.find_elements(By.CLASS_NAME, 'styles_businessUnitMain__PuwB7')

try:
    nom_entreprise_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/main/div/div[2]/div/section/div[4]/a/div/div[2]/p')
    nom_entreprise1 = nom_entreprise_element.text
except NoSuchElementException:
    nom_entreprise1 = 'NA'

try:
    trust_score_element = driver.find_element(By.CSS_SELECTOR, 'span.typography_body-m__xgxZ_')
    trust_score1 = trust_score_element.text.split()[-1]
except NoSuchElementException:
    trust_score1 = 'NA'

try:
    reviews_element = driver.find_element(By.CSS_SELECTOR, 'p.styles_ratingText__yQ5S7')
    reviews1 = reviews_element.text.split('|')[-1].strip()
except NoSuchElementException:
    reviews1 = 'NA'

try:
    emplacement_element1 = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/main/div/div[2]/div/section/div[4]/a/div/div[2]/div/span')
    emplacement1 = emplacement_element1.text
except NoSuchElementException:
    emplacement1 = 'NA'

if reviews1 == 'NA':
    trust_score1 = 'NA'

    
div_elements = driver.find_elements(By.XPATH, '//div[@class="paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2"]')
for div in div_elements:
    elements = div.find_element(By.CLASS_NAME, 'styles_footer__DoJVj')
    elements_2 = elements.find_element(By.CLASS_NAME, 'styles_wrapper___E6__')
    span_elements = elements_2.find_elements(By.TAG_NAME, 'span')
    concatenated_text = '' 
    for span in span_elements:
        concatenated_text += span.text
    
    result_list.append(concatenated_text)
    
df_new_column = pd.DataFrame(result_list, columns=['infos'])

driver.close()
driver.switch_to.window(driver.window_handles[0])

entreprise_elements = entreprise_elements + entreprise_elements_2

In [6]:
for entreprise in entreprise_elements[0:20]:
    
    try:
        nom_entreprise = entreprise.find_element(By.CSS_SELECTOR, 'p.styles_displayName__GOhL2').text
        
    except Exception as e:
        nom_entreprise = 'NA'

    try:
        trust_score_element = entreprise.find_element(By.CSS_SELECTOR, 'span.typography_body-m__xgxZ_')
        trust_score = trust_score_element.text.split()[-1]
    except Exception as e:
        trust_score = 'NA'

    try:
        reviews = entreprise.find_element(By.CSS_SELECTOR, 'p.styles_ratingText__yQ5S7').text.split('|')[-1].strip()
        
    except Exception as e:
        reviews = 'NA'

    try:
        emplacement = entreprise.find_element(By.CSS_SELECTOR, 'span.styles_location__ILZb0').text
    
    except Exception as e:
        emplacement = 'NA'

    if reviews == 'NA':
        trust_score = 'NA'
        
    informations_entreprises.append({
        'Nom': nom_entreprise,
        'TrustScore': trust_score,
        'Reviews': reviews,
        'Emplacement': emplacement,
    })

    
informations_entreprises.append({
    'Nom': nom_entreprise1,
    'TrustScore': trust_score1,
    'Reviews': reviews1,
    'Emplacement': emplacement1,
})




df_entreprises = pd.DataFrame(informations_entreprises)
df_entreprises = df_entreprises.assign(infos=df_new_column)

In [7]:
df_entreprises.head(5)

,Nom,TrustScore,Reviews,Emplacement,infos
0,Evergreen Credit Union,4.9,314 reviews,"Portland, United States",Credit Union·Financial Institution·ATM
1,Liberty First Credit Union,4.7,287 reviews,"Lincoln, United States",ATM·Mortgage Lender·Car Finance and Loan Company
2,GoHenry by Acorns,4.3,"3,179 reviews","Irvine, United States",Educational Institution·Financial Institution·ATM
3,NorthOne,4.3,180 reviews,"New York, United States",Money Transfer Service·Bookkeeping Service·
4,Crypto Dispensers,3.9,166 reviews,"Chicago, United States",ATM·Money Transfer Service·Investment Service


In [8]:
# Convert and write JSON object to file
#df_entreprises.to_csv('df_enterprises.csv',sep='|', index=False)

In [9]:
# Reccuperons les differents liens 

elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paper_paper__1PY90.paper_outline__lwsUX.card_card__lQWDv.card_noPadding__D8PcU.styles_wrapper__2JOo2'))
)
liens = []

next_page_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
next_page_link = next_page_button.get_attribute('href')

driver.execute_script("window.open('{}', '_blank');".format(next_page_link))
driver.switch_to.window(driver.window_handles[-1])

elements_1 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[4]/a")
for element in elements_1:
    href = element.get_attribute('href')
    liens.append(href)
elements_2 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[5]/a")
for element in elements_2:
    href = element.get_attribute('href')
    liens.append(href)



driver.close()
driver.switch_to.window(driver.window_handles[0])

for element in elements:
    link = element.find_element(By.TAG_NAME, 'a')
    href = link.get_attribute('href')
    liens.append(href)
    

In [10]:
# Attendre que tous les éléments avec la classe spécifique soient présents dans le DOM
elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paper_paper__1PY90.paper_outline__lwsUX.card_card__lQWDv.card_noPadding__D8PcU.styles_wrapper__2JOo2'))
)

score_etoile = []
commentaires_par_agence = {}

for element in elements:
    link = element.find_element(By.TAG_NAME, 'a')
    href = link.get_attribute('href')

    driver.execute_script("window.open('{}', '_blank');".format(href))

    driver.switch_to.window(driver.window_handles[-1])
    
    score_5_etoile = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/main/div/div[4]/section/div[2]/div[2]/label[1]/p[2]')
    score_5_etoile = score_5_etoile.text
    score_etoile.append(score_5_etoile)
    
    titre = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/main/div/div[3]/div[2]/div/div/div/section/div[1]/div[2]/h1/span[1]'))
        )
    titre_text = titre.text.strip()

    comments = driver.find_elements(By.CSS_SELECTOR, '.styles_reviewCardInner__EwDq2')
    
    if titre_text not in commentaires_par_agence:
        commentaires_par_agence[titre_text] = []
        
    if comments:
    
        for comment in comments:
            try:
                name_element = driver.find_element(By.CSS_SELECTOR, 'span[data-consumer-name-typography]')
                user_name = name_element.text
            except Exception as e:
                user_name = 'NA' 
            try:
                location_element = driver.find_element(By.CSS_SELECTOR, '[data-consumer-country-typography] > span')
                user_location = location_element.text
            except Exception as e:
                user_location = 'NA' 
            try:
                review_content_element = driver.find_element(By.CSS_SELECTOR, 'div[data-review-content]')
                review_content = review_content_element.text
            except Exception as e:
                review_content = 'NA' 
            try:
                title = comment.find_element(By.CSS_SELECTOR, '.typography_heading-s__f7029').text
            except Exception as e:
                title = 'NA'          
            try:
                review_text = comment.find_element(By.CSS_SELECTOR, '.typography_body-l__KUYFJ').text
            except Exception as e:
                review_text = 'NA'          
            try:
                review_count_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.styles_consumerExtraDetails__fxS4S > span.typography_body-m__xgxZ_'))
                )
                review_count = review_count_element.text
            except NoSuchElementException:
                review_count = 'NA'       
            try:
                date_of_experience = comment.find_element(By.CSS_SELECTOR, '.typography_body-m__xgxZ_').text
            except Exception as e:
                date_of_experience = 'NA'
                
            try:
                reply_element = comment.find_element(By.CSS_SELECTOR, '.paper_paper__1PY90')
                reply_text = reply_element.find_element(By.CSS_SELECTOR, '.styles_message__shHhX').text
                reply_date_element = reply_element.find_element(By.CSS_SELECTOR, '.styles_replyDate__Iem0_')
                reply_date = reply_date_element.get_attribute('title')
                
                comment_data = {
                    'User': user_name,
                    'localisation': user_location,
                    'Titre': title,
                    'commentaire': review_text,
                    'nombre_reviews': review_count,
                    'date_experience': date_of_experience,
                    'reply': {
                        'reply_text': reply_text,
                        'reply_date': reply_date
                    }
                }
            except NoSuchElementException:
                comment_data = {
                    'User': user_name,
                    'localisation': user_location,
                    'Titre': title,
                    'commentaire': review_text,
                    'nombre_reviews': review_count,
                    'date_experience': date_of_experience,
                    'reply': None
                }

            # Ajouter les données du commentaire (avec ou sans réponse) à la liste des commentaires de l'agence
            commentaires_par_agence[titre_text].append(comment_data)
            
    driver.close()

    driver.switch_to.window(driver.window_handles[0])
    
score_etoile.append(score_5_etoile)

In [12]:
df_new_column = pd.DataFrame(score_etoile, columns=['score_5_etoile'])
df_entreprises = df_entreprises.assign(score_5_etoile=df_new_column)
df_entreprises['Reviews'] = df_entreprises['Reviews'].apply(lambda x: x[:-8])
df_entreprises.head(5)

,Nom,TrustScore,Reviews,Emplacement,infos,score_5_etoile
0,Evergreen Credit Union,4.9,314,"Portland, United States",Credit Union·Financial Institution·ATM,96%
1,Liberty First Credit Union,4.7,287,"Lincoln, United States",ATM·Mortgage Lender·Car Finance and Loan Company,92%
2,GoHenry by Acorns,4.3,"3,179","Irvine, United States",Educational Institution·Financial Institution·ATM,66%
3,NorthOne,4.3,180,"New York, United States",Money Transfer Service·Bookkeeping Service·,54%
4,Crypto Dispensers,3.9,166,"Chicago, United States",ATM·Money Transfer Service·Investment Service,59%


In [13]:

df_entreprises.to_csv('../data/df_enterprises.csv',sep='|', index=False)


In [14]:

# Convert and write JSON object to file
with open("../data/df_commentaires_par_agence.json", "w") as outfile: 
    json.dump(commentaires_par_agence, outfile, indent = 4)


In [16]:
for titre_agence, commentaires in commentaires_par_agence.items():
    print(f"Titre d'agence: {titre_agence}")
    for commentaire in commentaires:
        print(commentaire)
    print("=============================")
    print("=============================")
    print("=============================")
    print("=============================")
    print("=============================")

Titre d'agence: Evergreen Credit Union
{'User': 'Donna Lamb', 'localisation': 'US', 'Titre': 'Best Credit Union', 'commentaire': "I have been with Evergreen Credit Union for many, many years and look forward to many more years. in all of those years everyone associated with Evergreen has been helpful and a pleasure to do business with and very, very pleasant. This includes everyone tellers, customer relations people, mid level management, top dogs. I could thank a lot of the staff by name at the credit union but it would include everyone. I would like to congratulate Evergreen Credit Union for building such a fine organization. Evergreen Credit Union is a wonderful partner and participant in the neighborhood and beyond. Kudo's Evergreen keep up the good work.", 'nombre_reviews': '2 reviews', 'date_experience': '2 reviews', 'reply': {'reply_text': "Hi Donna,\n\nThank you very much for the kind words! We're so happy to have built a credit union that can serve community members just like 

In [15]:
driver.close()